In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

Necessary imports

In [0]:
import sys
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import io
from __future__ import absolute_import, division, print_function, unicode_literals
import functools
import pandas as pd
from tensorflow.keras import regularizers

Upload heart_train.csv and heart_test.csv

In [6]:
from google.colab import files
uploaded = files.upload()

Saving heart_test.csv to heart_test.csv
Saving heart_train.csv to heart_train.csv


Parse training data and fit model

In [169]:
print("--Process data--")
train_data = pd.read_csv('heart_train.csv')

train_data['famhist'] = pd.Categorical(train_data['famhist'])
train_data['famhist'] = train_data.famhist.cat.codes

target = train_data.pop('chd')
dataset = tf.data.Dataset.from_tensor_slices((train_data.values, target.values))
train_dataset = dataset.shuffle(len(train_data)).batch(1)

print("--Make model--")
model = tf.keras.Sequential([
  tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.004)),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.004)),
  tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.004)),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])

print("--Fit model--")
model.fit(train_dataset, epochs=20, verbose=2)

--Process data--
--Make model--
--Fit model--

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/20
385/385 - 1s - loss: 6.0604 - accuracy: 0.5169
Epoch 2/20
385/385 - 1s - loss: 2.9909 - accuracy: 0.5714
Epoch 3/20
385/385 - 1s - loss: 2.2252 - accuracy: 0.5351
Epoch 4/20
385/385 - 1s - loss: 1.3450 - accuracy: 0.6234
Epoch 5/20
385/385 - 1s - loss: 1.4010 - accuracy: 0.6494
Epoch 6/20
385/385 - 1s - loss: 1.2124 - accuracy: 0.6182
Epoch 7/20
385/385 - 1s - loss: 1.2400 - accuracy: 0.6234
Epoch 8/20
385/385 - 1s - loss: 1.1890 - accuracy: 0.646

Parse test data and evaluate model

In [170]:
test_data = pd.read_csv('heart_test.csv')
test_data['famhist'] = pd.Categorical(test_data['famhist'])
test_data['famhist'] = test_data.famhist.cat.codes

target2 = test_data.pop('chd')

dataset2 = tf.data.Dataset.from_tensor_slices((test_data.values, target2.values))

test_dataset = dataset2.shuffle(len(test_data)).batch(1)

print("--Evaluate model--")
model_loss, model_acc = model.evaluate(test_dataset, verbose=2)
print(f"Model Loss:    {model_loss:.2f}")
print(f"Model Accuracy: {model_acc*100:.1f}%")

--Evaluate model--
77/77 - 0s - loss: 1.0487 - accuracy: 0.6753
Model Loss:    1.05
Model Accuracy: 67.5%
